In [27]:
import pandas as pd
import numpy as np

#Used to allow the browser to load. 
import time
from tqdm import tqdm

#used as a import for dateing the CSV
import datetime

#NLP libaries
import spacy
from spacy.lang.en import STOP_WORDS

import gensim
import pyLDAvis.gensim
from gensim.models import LdaModel
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import coherencemodel
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

## Using Topic Modeling using Gensim
Objective: Setup for topic modeling and use LDA to determine feature importance

In [28]:
job_list = pd.read_csv('./data/2018-07-03_Analyst_Austin, Texas job_list.csv')

In [29]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [30]:
pos_description = job_list['position_description'].values

In [31]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(pos_description),deacc=True))

In [32]:
data_words = list(sent_to_words(pos_description))

In [33]:
#Building bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=4, threshold=100)

trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

In [34]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [45]:
def remove_stopwords(text):
    return [[word for word in simple_preprocess(str(doc)) if word not in STOP_WORDS] for doc in text]

def make_bigrams(text):
    return [bigram_mod[doc] for doc in text]

def make_trigrams(text):
    return [trigram_mod[bigram_mod[doc]] for doc in text]

def lemmatization(text):
    text_rem = []
    for sent in text:
        doc = nlp(" ".join(sent))
        text_rem.append([token.lemma_ for token in doc])
        return text_rem 

In [64]:
remove_more_words = ['skill','datum','day','week','year','ability','analyst','recovery','austin','document','ago','business','Texas','job','apply','tx','experience']

In [65]:
words_no_stops = remove_stopwords(data_words)

word_bigrams = make_bigrams(words_no_stops)

data_lemma =  lemmatization(word_bigrams)

In [66]:
data_lemma[0] = [word for word in data_lemma[0] if word not in remove_more_words]

In [67]:
id2word = corpora.Dictionary(data_lemma)

texts = data_lemma

corpus = [id2word.doc2bow(text) for text in texts]

In [68]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word,num_topics=5,
                               random_state=42,
                               update_every=1,
                               chunksize=100,
                               passes=10,
                               alpha='auto',
                               per_word_topics=False)

In [69]:
doc_lda = lda_model[corpus]

In [70]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = coherencemodel.CoherenceModel(model=lda_model, texts=data_lemma , dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.307389928575341

Coherence Score:  0.30116296944834375


In [71]:
pyLDAvis.gensim.prepare(lda_model,corpus, id2word)

PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
2      99.995628        1       1 -0.141828  0.0
3       0.001181        1       2 -0.013123 -0.0
1       0.001098        1       3  0.043898  0.0
4       0.001056        1       4  0.053387 -0.0
0       0.001028        1       5  0.057665  0.0, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
3435  Default  468.000000           work  468.000000  30.0000  30.0000
543   Default  300.000000           team  300.000000  29.0000  29.0000
2790  Default  233.000000        project  233.000000  28.0000  28.0000
1895  Default  207.000000       analysis  207.000000  27.0000  27.0000
173   Default  236.000000        process  236.000000  26.0000  26.0000
2575  Default  183.000000           data  183.000000  25.0000  25.0000
1910  Default  218.000000        company  218.000000  24.0000  24.0000
439   Default  203.000000         report  203.000000  23.0000  23.0000
3063  Default  206.000000        include  206.000000  22.0000  22.0000
599   Default  202.000000        support  202.000000  21.0000  21.0000
961   Default  190.000000     management  190.000000  20.0000  20.0000
3569  Default  183.000000    requirement  183.000000  19.0000  19.0000
1510  Default  190.000000         system  190.000000  18.0000  18.0000
2548  Default  197.000000         easily  197.000000  17.0000  17.0000
2522  Default  160.000000        provide  160.000000  16.0000  16.0000
3480  Default  162.000000    application  162.000000  15.0000  15.0000
2525  Default  148.000000      knowledge  148.000000  14.0000  14.0000
1673  Default  171.000000         client  171.000000  13.0000  13.0000
3211  Default  141.000000        develop  141.000000  12.0000  12.0000
3311  Default  142.000000    information  142.000000  11.0000  11.0000
3351  Default  144.000000     technology  144.000000  10.0000  10.0000
3170  Default  157.000000        service  157.000000   9.0000   9.0000
801   Default  126.000000        require  126.000000   8.0000   8.0000
766   Default  142.000000    opportunity  142.000000   7.0000   7.0000
2768  Default  132.000000       customer  132.000000   6.0000   6.0000
1738  Default  132.000000           like  132.000000   5.0000   5.0000
313   Default  128.000000        product  128.000000   4.0000   4.0000
404   Default  146.000000      financial  146.000000   3.0000   3.0000
335   Default  136.000000      operation  136.000000   2.0000   2.0000
3364  Default  148.000000       solution  148.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
107    Topic5    0.000088        visitor    1.176544   1.9874  -8.2700
543    Topic5    0.000198           team  300.948425  -2.7506  -7.4637
439    Topic5    0.000178         report  203.398392  -2.4620  -7.5668
2525   Topic5    0.000169      knowledge  148.640656  -2.2021  -7.6206
2548   Topic5    0.000175         easily  197.028168  -2.4507  -7.5874
599    Topic5    0.000174        support  202.662567  -2.4816  -7.5900
2790   Topic5    0.000174        project  233.714371  -2.6253  -7.5912
1895   Topic5    0.000166       analysis  207.946976  -2.5581  -7.6408
1910   Topic5    0.000164        company  218.168365  -2.6144  -7.6491
173    Topic5    0.000164        process  236.065140  -2.6934  -7.6493
1673   Topic5    0.000157         client  171.610550  -2.4200  -7.6948
961    Topic5    0.000156     management  190.733032  -2.5342  -7.7033
3063   Topic5    0.000156        include  206.411423  -2.6094  -7.6995
1897   Topic5    0.000144       software  101.830856  -1.9807  -7.7774
335    Topic5    0.000149      operation  136.220245  -2.2433  -7.7490
2522   Topic5    0.000148        provide  160.321503  -2.4116  -7.7544
3569   Topic5    0.000150    requirement  183.764542  -2.5357  -7.7420
3480   Topic5    0.000147    application  162.486252  -2.4290  -